In [5]:
import math
import numpy as np
from typing import List, Set, Dict, Tuple, Optional


In [6]:

maxAge = 100
maxDurPregnant = 300
class MalePop:
    impregnationsPerDay: int = 3
    age: List[int] = [0] * maxAge

class FemalePop:
    def __init__(self) -> None:
        self.females = []
        for i in range(maxAge):
            self.females.append([0]*maxDurPregnant)
        
    females: List[List[int]] # age,durUntilGivingBirth 

In [7]:
import pandas as pd
csvPops = pd.read_csv("WORLD-2040.csv")
(csvPopsAge,csvPopsFemale) = (csvPops["Age"],csvPops["F"])

def initWoman(csvPopsAge,csvPopsFemale):
    femalePop: FemalePop = FemalePop()
    for i in range(len(csvPopsAge)):
        ages = csvPopsAge[i].split("-")
        if len(ages) != 2:
            break
        ages[0] = int(ages[0])
        ages[1] = int(ages[1])
        num = int(csvPopsFemale[i])
        numAges = ages[1] - ages[0] + 1
        numPerAge  = int(num/numAges)
        #print(numPerAge)
        for ageI in range(numAges):
            age = ages[0] + ageI
            femalePop.females[age][0] =numPerAge
    
    return femalePop


def initMen():
    malePop: MalePop = MalePop()
    malePop.age[20] = 2
    malePop.age[21] = 1
    malePop.age[23] = 1
    malePop.age[25] = 2
    malePop.age[31] = 1
    malePop.age[45] = 1
    return malePop
    

In [8]:
malePop: MalePop = initMen()
femalePop: FemalePop  = initWoman(csvPopsAge,csvPopsFemale)
reproductionStartAge = 18
reproductionEndAge = 50
#{pregnant: [0,100], age: [20,30]}

In [9]:
def impregnateHornyFemales(femalePop: FemalePop, num: int ):
    todo = num
    for age in range(reproductionStartAge,reproductionEndAge):
        if todo == 0:
            break
        females = femalePop.females[age][0]
        femalePop.females[age][0] -= todo
        if femalePop.females[age][0] < 0:
            todo = - femalePop.females[age][0]
            femalePop.females[age][0] = 0
        else:
            females = todo
            todo = 0
        
        avgDurPregnancy = 30 * 9 # TODO make exp
        femalePop.females[age][avgDurPregnancy] = females
        
    return

def updatePregnancies(femalePop: FemalePop):
    for age in range(len(femalePop.females)): 
        femalePop.females[age][0] += femalePop.females[age][1]
        pregnanciesFinished = femalePop.females[age][1]
        del femalePop.females[age][1]
        femalePop.females[age].append(0)
        if(pregnanciesFinished != 0):
            #print("Born:",pregnanciesFinished)
            femalePop.females[0][0] += pregnanciesFinished/2
            malePop.age[0] += pregnanciesFinished/2
        #for days in range(len(femalePop.females[age])-1,0,-1):
        #    femalePop.females[age][days] = femalePop.females[age][days-1]

def killHumans(malePop: MalePop,femalePop: FemalePop):
    #malePop.age =malePop.age[0:100]
    femalePop.females =femalePop.females[0:100]
    # TODO
    
def ageEveryone(malePop: MalePop,femalePop: FemalePop):
    malePop.age.insert(0,0)
    #del malePop.age[-1]
      
    femalePop.females.insert(0,[0]*maxDurPregnant)
    #del femalePop.females[-1]



In [10]:
def update(malePop: MalePop,femalePop: FemalePop, iter , waitingTime = 4 * 365):
    if waitingTime < iter:
        toImpregnate = 0
        for i in range(reproductionStartAge,len(malePop.age)):
            toImpregnate += malePop.age[i] * malePop.impregnationsPerDay 
        impregnateHornyFemales(femalePop,toImpregnate)
        #impregnateHornyFemales(femalePop,1)
        #print("Imp", toImpregnate)
    
    updatePregnancies(femalePop)
    if(iter%360 == 0):
        ageEveryone(malePop,femalePop)
        killHumans(malePop,femalePop)
    
        


In [11]:
def getNumberWoman(femalePop: FemalePop):
    num = 0
    for age in range(len(femalePop.females)):
        for i in range(len(femalePop.females[age])):
            num += femalePop.females[age][i]
    return num

def getNumberMan(malePop: MalePop):
    num = 0
    for age in malePop.age:
        num += age
    return num

def getNumberHumans(malePop: MalePop, femalePop: FemalePop):
    return getNumberMan(malePop) + getNumberWoman(femalePop)


In [12]:
x = []
yF = []
yM = []
for i in range(365*50):
    if(i%360 == 0):
        x.append(i/360)
        yF.append(getNumberWoman(femalePop))
        yM.append(getNumberMan(malePop))
        #print(i,getNumberWoman(femalePop))
    update(malePop,femalePop,i)

In [14]:
import plotly.express as px
fig = px.bar(x=x, y=yF)
#fig = px.bar(x=x, y=yM)
fig.show()